<a href="https://colab.research.google.com/github/Edfred1/Contextual-Music-Crafter/blob/main/CMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation and Setup

This section covers cloning the repository and installing the necessary dependencies.

In [ ]:
# Clone the Git repository
!git clone https://github.com/Edfred1/Contextual-Music-Crafter.git
%cd Contextual-Music-Crafter

# Install dependencies
!pip install -r requirements.txt

### Configuration

Here, we configure the `config.yaml` file. This file contains settings for the AI API, musical parameters, and instrument definitions. You'll need to replace `"YOUR_GOOGLE_AI_API_KEY"` with your actual Google AI API key.

In [ ]:
from ruamel.yaml import YAML
from pathlib import Path

# Path to config.yaml
config_path = Path('config.yaml')

# Load existing config (preserve comments); create a minimal starter if missing
yaml = YAML(typ='rt')
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

doc = None
if config_path.exists():
    try:
        with config_path.open('r', encoding='utf-8') as f:
            doc = yaml.load(f)
    except Exception:
        doc = None

if doc is None:
    doc = yaml.load("""
api_key:
  - "YOUR_GOOGLE_AI_API_KEY_1"
model_name: "gemini-2.5-pro"
temperature: 0
lyrics_temperature: 0.0
enable_hotkeys: 1
stage2_invalid_retries: 5
pass_raw_prompt_to_stages: 0
inspiration: "A new track"
genre: "House"
bpm: 125
key_scale: "C minor"
automation_settings:
  use_pitch_bend: 0
  use_sustain_pedal: 0
  use_cc_automation: 0
  allowed_cc_numbers: [1, 10, 11, 74]
max_output_tokens: 65536
context_window_size: -1
use_call_and_response: 0
number_of_iterations: 1
time_signature:
  beats_per_bar: 4
  beat_value: 4
instruments:
  - name: "Drums"
    program_num: 10
    role: "drums"
  - name: "Bass"
    program_num: 39
    role: "bass"
""")

# Helper function to convert to 0/1
def _to01(x):
    try:
        return 1 if int(x) == 1 else 0
    except Exception:
        return 0

# Helper function to convert to int
def _toint(x, default):
    try:
        return int(x)
    except Exception:
        return default

# Helper function to convert to float
def _tofloat(x, default):
    try:
        return float(x)
    except Exception:
        return default

print("=" * 70)
print("CONFIGURATION SETUP")
print("=" * 70)
print("\nSelect configuration mode:")
print("  1 = Quick Mode (essential settings only)")
print("  2 = Full Mode (all options with explanations)")
print("  3 = Skip (only API key required)")
mode = input("\nChoose mode [1/2/3] (default: 1): ").strip() or "1"

if mode == "1":
    # Quick Mode - essential settings only
    print("\n" + "=" * 70)
    print("QUICK MODE - Essential Settings")
    print("=" * 70)
    print("\n💡 Tip: Press Enter to keep current/default values\n")
    
    # API Key
    print("API Key (required)")
    print("  └─ Your Google AI API key(s) for authentication.")
    print("     Multiple keys can be comma-separated for automatic rotation.\n")
    api_keys_input = input(f"Enter Google AI API key(s): ").strip()
    if api_keys_input:
        keys = [k.strip() for k in api_keys_input.split(',') if k.strip()]
        doc['api_key'] = keys
    
    # Model
    print("\nModel Name")
    print("  └─ AI model to use (e.g., 'gemini-2.5-pro' or 'gemini-2.5-flash').")
    print("     Pro models are higher quality but slower/more expensive.\n")
    model = input(f"Model name [{doc.get('model_name','gemini-2.5-pro')}]: ").strip()
    if model:
        doc['model_name'] = model
    
    # Genre
    print("\nGenre")
    print("  └─ Musical genre/style (e.g., 'House', 'Techno', 'Rock', 'Jazz').")
    print("     This influences the overall musical character.\n")
    genre = input(f"Genre [{doc.get('genre','House')}]: ").strip()
    if genre:
        doc['genre'] = genre
    
    # BPM
    print("\nBPM (Beats Per Minute)")
    print("  └─ Tempo of the track (e.g., 120 for slow, 140 for fast).")
    print("     Common: House=120-130, Techno=130-140, Dubstep=140-150.\n")
    bpm = input(f"BPM [{doc.get('bpm',125)}]: ").strip()
    if bpm:
        doc['bpm'] = _toint(bpm, doc.get('bpm',125))
    
    # Key and scale
    print("\nKey & Scale")
    print("  └─ Musical key and scale (e.g., 'C major', 'A minor', 'F# dorian').")
    print("     Determines the harmonic foundation of the track.\n")
    key_scale = input(f"Key and scale [{doc.get('key_scale','C minor')}]: ").strip()
    if key_scale:
        doc['key_scale'] = key_scale
    
    # Instruments (optional)
    instruments = doc.get('instruments', [])
    if instruments:
        print(f"\nCurrent Instruments ({len(instruments)}):")
        for i, inst in enumerate(instruments):
            print(f"  {i+1}. {inst.get('name','Unknown')}")
    
    edit_instruments = input("\nEdit instruments? (y/N): ").strip().lower()
    if edit_instruments == 'y':
        print("\nInstrument Configuration")
        print("  └─ Define which instruments play in your track.")
        print("     Each instrument has a name, MIDI program number (1-128), and role.\n")
        print("Current instruments:")
        for i, inst in enumerate(instruments):
            print(f"  {i+1}. {inst.get('name','Unknown')} - Program {inst.get('program_num',0)} - Role: {inst.get('role','unknown')}")
        
        action = input("\nAdd new instrument? (y/N): ").strip().lower()
        if action == 'y':
            name = input("Instrument name: ").strip()
            if name:
                program = input("Program number (1-128): ").strip()
                role = input("Role (drums, bass, pads, lead, melody, etc.): ").strip()
                if program and role:
                    try:
                        new_inst = {
                            'name': name,
                            'program_num': _toint(program, 1),
                            'role': role
                        }
                        instruments.append(new_inst)
                        doc['instruments'] = instruments
                        print(f"✓ Added instrument: {name}")
                    except Exception as e:
                        print(f"✗ Error: {e}")
        
        if instruments:
            remove = input("\nRemove instrument? (y/N): ").strip().lower()
            if remove == 'y':
                try:
                    idx = int(input(f"Enter number (1-{len(instruments)}): ").strip()) - 1
                    if 0 <= idx < len(instruments):
                        removed = instruments.pop(idx)
                        doc['instruments'] = instruments
                        print(f"✓ Removed: {removed.get('name','Unknown')}")
                except Exception:
                    print("✗ Invalid input.")

elif mode == "2":
    # Full Mode - all options with explanations
    print("\n" + "=" * 70)
    print("FULL MODE - All Configuration Options")
    print("=" * 70)
    print("\n💡 Tip: Press Enter at each prompt to keep the default value\n")
    
    print("=" * 70)
    print("API CONFIGURATION")
    print("=" * 70)
    
    print("\nAPI Key (required)")
    print("  └─ Your Google AI API key(s) for authentication.")
    print("     Multiple keys can be comma-separated for automatic rotation on quota errors.\n")
    api_keys_input = input(f"Enter Google AI API key(s): ").strip()
    if api_keys_input:
        keys = [k.strip() for k in api_keys_input.split(',') if k.strip()]
        doc['api_key'] = keys
    
    print("\nModel Name")
    print("  └─ AI model to use: 'gemini-2.5-pro' (best quality) or 'gemini-2.5-flash' (faster/cheaper).\n")
    model = input(f"Model name [{doc.get('model_name','gemini-2.5-pro')}]: ").strip()
    if model:
        doc['model_name'] = model
    
    print("\nTemperature")
    print("  └─ Controls randomness: 0.0 = deterministic/consistent, 2.0 = very creative/variable.")
    print("     Lower values are more reliable but less creative.\n")
    temp = input(f"Temperature [{doc.get('temperature',0)}]: ").strip()
    if temp:
        doc['temperature'] = _tofloat(temp, doc.get('temperature',0))
    
    print("\nLyrics Temperature")
    print("  └─ Randomness for lyrics generation (if applicable).")
    print("     Separate from main temperature for fine-tuning lyrics creativity.\n")
    lyrics_temp = input(f"Lyrics temperature [{doc.get('lyrics_temperature',0.0)}]: ").strip()
    if lyrics_temp:
        doc['lyrics_temperature'] = _tofloat(lyrics_temp, doc.get('lyrics_temperature',0.0))
    
    print("\nEnable Hotkeys")
    print("  └─ Enable runtime hotkeys (1=on, 0=off) for model switching/backoff controls.")
    print("     Useful for long runs where you might need to adjust on the fly.\n")
    hotkeys = input(f"Enable hotkeys (1=on, 0=off) [{doc.get('enable_hotkeys',1)}]: ").strip()
    if hotkeys:
        doc['enable_hotkeys'] = _to01(hotkeys)
    
    print("\nStage-2 Invalid Retries")
    print("  └─ Number of retries when Stage-2 (notes composition) produces invalid output.")
    print("     Quota/429 errors don't count toward this limit.\n")
    retries = input(f"Stage-2 invalid retries [{doc.get('stage2_invalid_retries',5)}]: ").strip()
    if retries:
        doc['stage2_invalid_retries'] = _toint(retries, doc.get('stage2_invalid_retries',5))
    
    print("\nPass Raw Prompt to Stages")
    print("  └─ Whether to pass the raw user prompt to Stage-1/2 (1=yes, 0=no).")
    print("     If 0, only distilled info is passed (lower risk of meta-leaks).\n")
    raw_prompt = input(f"Pass raw prompt to stages (1=yes, 0=no) [{doc.get('pass_raw_prompt_to_stages',0)}]: ").strip()
    if raw_prompt:
        doc['pass_raw_prompt_to_stages'] = _to01(raw_prompt)
    
    print("\n" + "=" * 70)
    print("MUSICAL PARAMETERS")
    print("=" * 70)
    
    print("\nInspiration")
    print("  └─ Free-form description to guide the composition.")
    print("     This is the main creative prompt that influences the track's character.\n")
    insp = input(f"Inspiration (Enter to keep current): ").strip()
    if insp:
        doc['inspiration'] = insp
    
    print("\nGenre")
    print("  └─ Main musical style/genre (e.g., 'House', 'Techno', 'Rock', 'Jazz', 'Ambient').")
    print("     This sets the overall musical context.\n")
    genre = input(f"Genre [{doc.get('genre','House')}]: ").strip()
    if genre:
        doc['genre'] = genre
    
    print("\nBPM (Beats Per Minute)")
    print("  └─ Tempo: 60-80=slow, 120-130=medium, 140+=fast.")
    print("     Genre examples: House=120-130, Techno=130-140, Dubstep=140-150.\n")
    bpm = input(f"BPM [{doc.get('bpm',125)}]: ").strip()
    if bpm:
        doc['bpm'] = _toint(bpm, doc.get('bpm',125))
    
    print("\nKey & Scale")
    print("  └─ Musical key and scale (e.g., 'C major', 'A minor', 'F# dorian').")
    print("     Available: major, minor, dorian, phrygian, lydian, mixolydian, etc.\n")
    key_scale = input(f"Key and scale [{doc.get('key_scale','C minor')}]: ").strip()
    if key_scale:
        doc['key_scale'] = key_scale
    
    print("\n" + "=" * 70)
    print("AUTOMATION SETTINGS")
    print("=" * 70)
    print("\nMIDI automation controls for expressive performance.")
    print("These add pitch bends, sustain, and control changes to make the music more dynamic.\n")
    auto = doc.get('automation_settings') or {}
    if input("Edit automation settings? (y/N): ").strip().lower() == 'y':
        print("\nUse Pitch Bend")
        print("  └─ Enable pitch bend automation (1=on, 0=off) for slides and vibrato.\n")
        auto['use_pitch_bend'] = _to01(input(f"Use pitch bend (1=on, 0=off) [{auto.get('use_pitch_bend',0)}]: ") or auto.get('use_pitch_bend',0))
        
        print("\nUse Sustain Pedal")
        print("  └─ Enable sustain pedal (CC64) for legato/hold effects (1=on, 0=off).")
        print("     Useful for pianos and pads.\n")
        auto['use_sustain_pedal'] = _to01(input(f"Use sustain pedal (1=on, 0=off) [{auto.get('use_sustain_pedal',0)}]: ") or auto.get('use_sustain_pedal',0))
        
        print("\nUse CC Automation")
        print("  └─ Enable Control Change automation (1=on, 0=off) for filter/volume/pan curves.\n")
        auto['use_cc_automation'] = _to01(input(f"Use CC automation (1=on, 0=off) [{auto.get('use_cc_automation',0)}]: ") or auto.get('use_cc_automation',0))
        
        if auto.get('use_cc_automation',0) == 1:
            print("\nAllowed CC Numbers")
            print("  └─ CC numbers allowed for automation (comma-separated).")
            print("     Common: 1=Modulation, 10=Pan, 11=Expression, 74=Filter Cutoff.\n")
            cc_str = input(f"Allowed CC numbers [{auto.get('allowed_cc_numbers',[1,10,11,74])}]: ").strip()
            if cc_str:
                try:
                    auto['allowed_cc_numbers'] = [int(x.strip()) for x in cc_str.split(',') if x.strip()]
                except Exception:
                    pass
    doc['automation_settings'] = auto
    
    print("\n" + "=" * 70)
    print("GENERATION & PERFORMANCE SETTINGS")
    print("=" * 70)
    
    print("\nMax Output Tokens")
    print("  └─ Maximum tokens per model response (higher = longer outputs, more cost).")
    print("     Increase if responses get truncated; decrease to save tokens.\n")
    max_tokens = input(f"Max output tokens [{doc.get('max_output_tokens',65536)}]: ").strip()
    if max_tokens:
        doc['max_output_tokens'] = _toint(max_tokens, doc.get('max_output_tokens',65536))
    
    print("\nContext Window Size")
    print("  └─ Previous themes context: -1=dynamic (use as much as fits), 0=none, >0=fixed number.")
    print("     Higher values give more musical context but use more tokens.\n")
    ctx_size = input(f"Context window size (-1=dynamic, 0=none, >0=fixed) [{doc.get('context_window_size',-1)}]: ").strip()
    if ctx_size:
        doc['context_window_size'] = _toint(ctx_size, doc.get('context_window_size',-1))
    
    print("\nUse Call & Response")
    print("  └─ Enable call-and-response patterns for melodic instruments (1=yes, 0=no).")
    print("     Creates interactive musical phrases between instruments.\n")
    call_resp = input(f"Use call and response (1=yes, 0=no) [{doc.get('use_call_and_response',0)}]: ").strip()
    if call_resp:
        doc['use_call_and_response'] = _to01(call_resp)
    
    print("\nNumber of Iterations")
    print("  └─ Number of complete songs to generate per run.")
    print("     Each iteration creates a full track based on the configuration.\n")
    iterations = input(f"Number of iterations [{doc.get('number_of_iterations',1)}]: ").strip()
    if iterations:
        doc['number_of_iterations'] = _toint(iterations, doc.get('number_of_iterations',1))
    
    print("\n" + "=" * 70)
    print("TIME SIGNATURE")
    print("=" * 70)
    
    print("\nBeats Per Bar")
    print("  └─ Number of beats in each measure (e.g., 4 for 4/4 time, 3 for 3/4 time).\n")
    ts = doc.get('time_signature') or {}
    beats_per_bar = input(f"Beats per bar [{ts.get('beats_per_bar',4)}]: ").strip()
    if beats_per_bar:
        ts['beats_per_bar'] = _toint(beats_per_bar, ts.get('beats_per_bar',4))
    
    print("\nBeat Value")
    print("  └─ Note value that receives one beat (usually 4 for quarter note).\n")
    beat_value = input(f"Beat value [{ts.get('beat_value',4)}]: ").strip()
    if beat_value:
        ts['beat_value'] = _toint(beat_value, ts.get('beat_value',4))
    doc['time_signature'] = ts
    
    print("\n" + "=" * 70)
    print("INSTRUMENTS")
    print("=" * 70)
    print("\nDefine which instruments play in your track.")
    print("Each instrument needs: name, MIDI program number (1-128), and role.")
    print("Available roles: drums, bass, pads, chords, lead, melody, vocal, fx, etc.\n")
    instruments = doc.get('instruments', [])
    if instruments:
        print(f"Current instruments ({len(instruments)}):")
        for i, inst in enumerate(instruments):
            print(f"  {i+1}. {inst.get('name','Unknown')} - Program {inst.get('program_num',0)} - Role: {inst.get('role','unknown')}")
    
    if input("\nEdit instruments? (y/N): ").strip().lower() == 'y':
        action = input("Add new instrument? (y/N): ").strip().lower()
        if action == 'y':
            name = input("Instrument name: ").strip()
            if name:
                program = input("Program number (1-128): ").strip()
                role = input("Role (drums, bass, pads, lead, melody, etc.): ").strip()
                if program and role:
                    try:
                        new_inst = {
                            'name': name,
                            'program_num': _toint(program, 1),
                            'role': role
                        }
                        instruments.append(new_inst)
                        doc['instruments'] = instruments
                        print(f"✓ Added: {name}")
                    except Exception as e:
                        print(f"✗ Error: {e}")
        
        if instruments:
            remove = input("Remove instrument? (y/N): ").strip().lower()
            if remove == 'y':
                try:
                    idx = int(input(f"Enter number (1-{len(instruments)}): ").strip()) - 1
                    if 0 <= idx < len(instruments):
                        removed = instruments.pop(idx)
                        doc['instruments'] = instruments
                        print(f"✓ Removed: {removed.get('name','Unknown')}")
                except Exception:
                    print("✗ Invalid input.")

elif mode == "3":
    # Skip Mode - only API Key
    print("\n" + "=" * 70)
    print("SKIP MODE - API Key Only")
    print("=" * 70)
    print("\nOnly the API key will be updated. All other settings remain unchanged.\n")
    api_keys_input = input("Enter Google AI API key(s): ").strip()
    if api_keys_input:
        keys = [k.strip() for k in api_keys_input.split(',') if k.strip()]
        doc['api_key'] = keys
    print("\n✓ All other settings remain unchanged.")

# Save back preserving comments
with config_path.open('w', encoding='utf-8') as f:
    yaml.dump(doc, f)

print("\n" + "=" * 70)
print("✓ config.yaml updated successfully (comments preserved)")
print("=" * 70)

### Running a Python Program from the Repository

This cell lists the Python programs found in the cloned repository and prompts you to select one to execute. The selected program will then be run based on the configuration.

In [ ]:
import os

# Get a list of all files and directories in the cloned repository
files_and_directories = os.listdir('.')

# Filter for Python files
python_files = [f for f in files_and_directories if f.endswith('.py')]

if not python_files:
    print("No Python files found in the repository.")
else:
    print("Found Python files:")
    for i, filename in enumerate(python_files):
        print(f"{i+1}. {filename}")

    # Prompt the user to select a file
    while True:
        try:
            choice = int(input(f"Enter the number of the file to execute (1-{len(python_files)}): "))
            if 1 <= choice <= len(python_files):
                selected_file = python_files[choice - 1]
                print(f"Executing {selected_file}...")
                # Execute the selected Python file
                !python {selected_file}
                break
            else:
                print("Invalid input. Please enter a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")